In [1]:
import pandas as pd
import re 
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)


In [19]:
df = pd.read_csv("sample_data.csv")
exceptions = pd.merge(pd.read_csv("exceptions_countries.csv"),pd.read_csv("exсeptions_city.csv"), on='Сокращенное наименование')
text = 'Здравствуйте! В Санкт-Петербурге нет Генерального консульства Австралии.  По консульским вопросам можно обратиться в Посольство Австралии в Москве по телефону: +7 495 956-60-70. 24-часовая консульская помощь для граждан Австралии по телефону: +61 2 6261 3305.  Ограниченную экстренную консульскую помощь при определенных обстоятельствах может оказать Почетное консульство Австралии в Санкт-Петербурге по адресу: улица Мойка, дом 11. Тел.: +7 964 333 7572.  Почетное консульство не отвечает на визовые запросы.  С уважением, Комитета по внешним связям Санкт-Петербурга.'
def function(i):
    location_f = []
    i = re.sub(r'\[.*?\]', '', i)
    doc = Doc(i)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    location = list(filter(lambda x: x.type == 'LOC', doc.spans))
    for span in location:
        if span.normal not in exceptions['Сокращенное наименование'].values:
            location_f.append(span)
    location_f = [(span.text, span.tokens) for span in location_f]
    if not location_f:
        location_f = "None"
    return location_f
df['Data'] = df['Текст комментария'].map(function)
df
      

,Unnamed: 0,message_id,Дата и время,Текст комментария,Data
0,0,0,2023.01.26 16:32,Здравствуйте! В Санкт-Петербурге нет Генеральн...,"[(Санкт-Петербурга, [DocToken(start=551, stop=..."
1,1,1,2023.01.26 11:55,[club143265175|Центральный район Санкт-Петербу...,"[(ЖК, [DocToken(start=215, stop=217, text='ЖК'..."
2,2,2,2023.01.28 12:39,"1) Фурштатская, 19 Отслоение штукатурного слоя...",None
3,3,3,2023.01.28 12:42,"2) Фурштатская, 17 Здесь прямо-таки умоляю обр...",None
4,4,4,2023.01.28 12:45,"3) Фурштатская, 13 Отслоение штукатурного слоя...",None
...,...,...,...,...,...
195,195,195,2022.10.24 14:02,На Чайковского 63 тоже идет кап.ремонт. В квар...,None
196,196,196,2022.10.21 22:22,Вся улица Жуковского и Восстания заклеена рекл...,"[(ЖСК-1, [DocToken(start=107, stop=112, text='..."
197,197,197,2022.10.26 12:57,О каком благоустройстве идёт речь. Стремянная...,None
198,198,198,2022.10.20 22:08,🤣🤣🤣🤣 угол 1й советской и Суворовского Клюваты...,None


In [17]:
df.to_csv('save.csv', index=False)